In [9]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

Found existing installation: unsloth 2025.2.12
Uninstalling unsloth-2025.2.12:
  Successfully uninstalled unsloth-2025.2.12
  Cloning https://github.com/unslothai/unsloth.git (to revision nightly) to /tmp/pip-req-build-zmkvv6rs
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-zmkvv6rs
  Running command git checkout -b nightly --track origin/nightly
  Switched to a new branch 'nightly'
  Branch 'nightly' set up to track remote branch 'nightly' from 'origin'.
  Resolved https://github.com/unslothai/unsloth.git to commit 0c1a808e3a5828c615921fe7d3c8c10d7de6324c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-x6vz_uvz
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-req-build-x6vz_uvz
  Resolved https://g

In [1]:
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade datasets
!pip install trl
!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [8]:
import os
import torch
import pandas as pd
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from unsloth import is_bfloat16_supported

# LoRA hyperparameter grid
lora_r_values = [8, 16]
lora_alpha_values = [8]

# Load model and tokenizer
model_name = "unsloth/Llama-3.2-3B-Instruct"
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load dataset
dataset = load_dataset('csv', data_files="/content/dataset.csv", split='train[:]', trust_remote_code=True)

def format_prompt(examples, tokenizer):
    prompt_style = """
    You are a data analyst bot working with a database. The table nba_roster has the following columns and metadata:

    Team: A string containing the team name (e.g., "Atlanta Hawks", "Washington Wizards").
    NAME: A string containing the player's name (e.g., "Saddiq Bey", "Corey Kispert").
    Jersey: A string containing the player's jersey number (e.g., "1", "55").
    POS: A string containing the player's position (e.g., "SF", "SG", "G").
    AGE: An integer containing the player's age (e.g., 23, 31).
    HT: A string containing the player's height in feet and inches (e.g., "6'6", "6'2").
    WT: A string containing the player's weight in pounds (e.g., "180 lbs", "185 lbs").
    COLLEGE: A string containing the player's college name (e.g., "Utah", "Toledo").
    SALARY: A string containing the player's salary, formatted with a dollar sign (e.g., "$8,195,122", "$1,719,864").

    You must generate a response that is ONLY a JSON object. The JSON object must have exactly two keys: "question" and "sql".
    DO NOT include any extra text, explanations, or commentary outside the JSON object. The JSON must begin with '{{' and end with '}}' and nothing else.

    The output of your response is directly fed to another system, and no human is reading it. Providing anything other than JSON will cause the other system to crash.
    Format your response as follows:
    {{
      "question": "<The given question>",
      "sql": "<The SQL query>"
    }}

    Example:
    Question: What are all the rows of the table in the database?
    Response:
    {{
      "question": "What are all the rows of the table in the database?",
      "sql": "SELECT * FROM nba_roster;"
    }}

    REMEMBER: DO NOT YAP. DO NOT PROVIDE ANYTHING OTHER THAN THE JSON.
    Now, answer the following question:
    Question: {}
    SQL:
    """

    return {
        "text": [prompt_style.format(q) + tokenizer.eos_token for q in examples["question"]]
    }


results = []
best_loss = float("inf")
best_model_path = ""

model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit
        )

dataset = dataset.map(format_prompt, batched=True, fn_kwargs={"tokenizer": tokenizer})

for r in lora_r_values:
    for alpha in lora_alpha_values:
        print(f"Training with r={r}, alpha={alpha}...")

        model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)



        model = FastLanguageModel.get_peft_model(
            model,
            r=r,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            lora_alpha=alpha,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=3407,
            use_rslora=False,
            loftq_config=None,
        )

        training_args = TrainingArguments(
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=5,
            max_steps=60,
            learning_rate=1e-5,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            output_dir=f"outputs_r{r}_alpha{alpha}"
        )

        trainer = SFTTrainer(
            model=model,
            tokenizer=tokenizer,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=max_seq_length,
            dataset_num_proc=2,
            args=training_args,
        )

        trainer_stats = trainer.train()
        final_loss = trainer_stats.training_loss
        print(f"Final loss for r={r}, alpha={alpha}: {final_loss}")

        results.append({"r": r, "alpha": alpha, "loss": final_loss})

        model_path = f"fine-tuned-model_r{r}_alpha{alpha}"
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)

        if final_loss < best_loss:
            best_loss = final_loss
            best_model_path = model_path

# Print results table
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="loss"))

# Download best model
if best_model_path:
    print(f"Best model saved at: {best_model_path}")


==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Training with r=8, alpha=8...
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Tokenizing train dataset (num_proc=2):   0%|          | 0/313 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/313 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 313 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 12,156,928


Step,Training Loss
1,1.745000
2,1.749200
3,1.753700
4,1.734700
5,1.757600
6,1.745000
7,1.742000
8,1.737800
9,1.732400
10,1.722900


...

Tokenizing train dataset (num_proc=2):   0%|          | 0/313 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/313 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 313 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.745000
2,1.749200
3,1.753800
4,1.734900
5,1.757800
6,1.745500
7,1.743000
8,1.739000
9,1.734000
10,1.725200


Final loss for r=16, alpha=8: 1.6540373027324677
    r  alpha      loss
0   8      8  1.650211
1  16      8  1.654037
Best model saved at: fine-tuned-model_r8_alpha8
